In [1]:
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from rpy2.robjects import numpy2ri

import pandas as pd
import numpy as np

pandas2ri.activate()

/Users/danieloliveira/opt/anaconda3/envs/new_environment/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:17: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex


In [8]:
np.random.seed(1)

# environment 1
na = 140
X1a = 0.3 * np.random.randn(na)
X3a = X1a + 0.2 * np.random.randn(na)
Ya = -.7 * X1a + .6 * X3a + 0.1 * np.random.randn(na)
X2a = -0.5 * Ya + 0.5 * X3a + 0.1 * np.random.randn(na)

# environment 2
nb = 80
X1b = 0.3 * np.random.randn(nb)
X3b = 0.5 * np.random.randn(nb)
Yb = -.7 * X1b + .6 * X3b + 0.1 * np.random.randn(nb)
X2b = -0.5 * Yb + 0.5 * X3b + 0.1 * np.random.randn(nb)

# Combine environments
X1 = np.concatenate((X1a, X1b))
X2 = np.concatenate((X2a, X2b))
X3 = np.concatenate((X3a, X3b))
Y = np.concatenate((Ya, Yb))

Xmatrix = np.column_stack((X1, X2, X3))
Xmatrix_df = pd.DataFrame(Xmatrix, columns=['X1', 'X2', 'X3'])

# Convert numpy arrays to R objects
r_Xmatrix = numpy2ri.numpy2ri(Xmatrix)
r_Y = numpy2ri.numpy2ri(Y)

# Load seqICP library
robjects.r('''
    library(seqICP)
''')

# Run the seqICP analysis
robjects.globalenv['Xmatrix'] = r_Xmatrix
robjects.globalenv['Y'] = r_Y
robjects.r('''
    seqICP_result <- seqICP(X = Xmatrix, Y = Y, 
        par.test = list(grid = seq(0, length(Y), length(Y)/10), complements = FALSE, link = sum,
        alpha = 0.05, B = 100), max.parents = 4, stopIfEmpty = FALSE, silent = TRUE)
    seqICP_summary <- summary(seqICP_result)
    parent_set <- seqICP_result$parent.set

''')

# Retrieve the results
seqICP_summary = robjects.r['seqICP_summary']


 Invariant Linear Causal Regression at level 0.05

 Variables X1, X3 show a significant causal effect

 
 
         
 coefficient
 lower bound
 upper bound
  p-value
  

intercept
       -0.01
     -0.0430
      0.0286
       NA
  

X1       
       -0.73
     -0.7870
     -0.5064
     0.02
 *

X2       
        0.00
      0.0000
      0.0000
     0.53
  

X3       
        0.60
      0.5600
      0.7210
     0.02
 *


---
Signif. codes:  

0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1








In [9]:

# Retrieve the results
parent_set = robjects.r['parent_set']

parent_set

array([1, 3], dtype=int32)

In [11]:
Xmatrix_df.tail()

,X1,X2,X3
215,0.150518,0.329003,0.749054
216,-0.178003,-0.087175,-0.150488
217,0.153092,0.095745,0.403728
218,-0.274738,-0.084745,0.156119
219,-0.122176,0.031934,-0.096661


In [15]:
Xmatrix_df.columns[(parent_set - 1)]

Index(['X1', 'X3'], dtype='object')